# Botanical Research

In [1]:
import numpy as np
import cv2
import tensorflow
tensorflow.__version__

'2.5.0'

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, optimizers

In [3]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
images_path = r"C:\Users\ac253\AINML\Computer Vision Week 3 (Project 1)\train-20210824T073124Z-001.zip"

In [5]:
from zipfile import ZipFile
with ZipFile(images_path, 'r') as zip:
  zip.extractall()

In [6]:
import os
cwd=os.getcwd()
print(os.listdir(cwd))

['.ipynb_checkpoints', 'Computer vision project.ipynb', 'CV 1 - Project-1.pdf', 'Predict.png', 'Seedling - Prediction-20210824T073124Z-001.zip', 'train', 'train-20210824T073124Z-001.zip']


We can see that a folder called train has been generated.

## Training a Neural Network

In [7]:
def train_loop(lr,lamb,verb=True):
    learning_rate=1e-3
    lamb = 1e-5
    hidden_nodes=324
    output_nodes=12
    
    model=Sequential()
    model.add(Flatten())
    model.add(Dense(hidden_nodes, input_shape=(1024,),activation='relu'))
    model.add(Dense(hidden_nodes,activation='relu'))
    model.add(Dense(output_nodes,activation='softmax',kernel_regularizer=regularizers.l2(lamb)))
    
    sgd=optimizers.SGD(lr=learning_rate,decay=1e-6, momentum=0.9)
    
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    model.fit_generator(
    train_set,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_set, 
    validation_steps = validation_generator.samples // 32,
    epochs = 20)
    score=model.evaluate(train_generator,verbose=0)
    return score

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir=r"C:\Users\ac253\AINML\Computer Vision Week 3 (Project 1)\train"

# Create data generator for training data with data augmentation and normalizing all
# values by 255
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

# Setting training data generator's source directory
# Setting the target size to resize all the images to (64,64) as the model input layer expects 32X32 images

train_set = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(16, 16),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_set = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(16, 16),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 3816 images belonging to 12 classes.
Found 951 images belonging to 12 classes.


In [9]:
learning_rate=1e-3
lamb = 1e-5
hidden_nodes=324
output_nodes=12
    
model=Sequential()
model.add(Flatten())
model.add(Dense(hidden_nodes, input_shape=(768,),activation='relu'))
model.add(Dense(hidden_nodes,activation='relu'))
model.add(Dense(output_nodes,activation='softmax',kernel_regularizer=regularizers.l2(lamb)))

sgd=optimizers.SGD(lr=learning_rate,decay=1e-6, momentum=0.9)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
train_set,
steps_per_epoch = train_set.samples // 32,
validation_data = validation_set, 
validation_steps = validation_set.samples // 32,
epochs = 20)

score=model.evaluate(train_set,verbose=0)

Epoch 1/20
119/119 [==============================] - 41s 340ms/step - loss: 2.4196 - accuracy: 0.1535 - val_loss: 2.3995 - val_accuracy: 0.1412
Epoch 2/20
119/119 [==============================] - 38s 321ms/step - loss: 2.3881 - accuracy: 0.1797 - val_loss: 2.3824 - val_accuracy: 0.1552
Epoch 3/20
119/119 [==============================] - 39s 324ms/step - loss: 2.3741 - accuracy: 0.1945 - val_loss: 2.3631 - val_accuracy: 0.2177
Epoch 4/20
119/119 [==============================] - 39s 330ms/step - loss: 2.3537 - accuracy: 0.2080 - val_loss: 2.3503 - val_accuracy: 0.1649
Epoch 5/20
119/119 [==============================] - 39s 324ms/step - loss: 2.3341 - accuracy: 0.2146 - val_loss: 2.3229 - val_accuracy: 0.2274
Epoch 6/20
119/119 [==============================] - 39s 327ms/step - loss: 2.3122 - accuracy: 0.2289 - val_loss: 2.3057 - val_accuracy: 0.2338
Epoch 7/20
119/119 [==============================] - 38s 325ms/step - loss: 2.2891 - accuracy: 0.2368 - val_loss: 2.2838 - val_ac

In [10]:
test_image1 = cv2.imread(r'C:\Users\ac253\AINML\Computer Vision Week 3 (Project 1)\Predict.png')
# Resize the image to 64X64 shape to be compatible with the model
test_image1 = cv2.resize(test_image1,(16,16))

# Check if the size of the Image array is compatible with Keras model
print(test_image1.shape)

# If not compatible expand the dimensions to match with the Keras Input
test_image1 = np.expand_dims(test_image1, axis = 0)
test_image1 =test_image1*1/255.0

#Check the size of the Image array again
print('After expand_dims: '+ str(test_image1.shape))


#Predict the result of the test image
result = model.predict(test_image1)

# Check the indices Image Data Generator has allotted to each folder
classes_dict = train_set.class_indices
print(classes_dict)

# Creating a list of classes in test set for showing the result as the folder name
prediction_class = []
for class_name,index in classes_dict.items():
  prediction_class.append(class_name)
  
print(result[0])

# Index of the class with maximum probability
predicted_index = np.argmax(result[0])

# Print the name of the class
print(prediction_class[predicted_index])

(16, 16, 3)
After expand_dims: (1, 16, 16, 3)
{'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3, 'Common wheat': 4, 'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8, 'Shepherds Purse': 9, 'Small-flowered Cranesbill': 10, 'Sugar beet': 11}
[0.07028779 0.53341454 0.03721414 0.02631604 0.04447062 0.04294013
 0.00817322 0.02281118 0.03006822 0.014699   0.12303982 0.04656532]
Charlock


## Training a CNN

In [11]:
#Building the CNN clasifier.
classifier=Sequential()

# Adding a conv layer with 32 kernels of 3x3 shape and activation function Relu
classifier.add(Conv2D(32,(3,3), input_shape =(64,64,3),activation ='relu', padding = 'same'))

# Adding a maxpooling layer of size 2x2
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding a maxpooling layer of size 2x2
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Add another Convolution layer with 32 kernels of 3X3 shape with activation function ReLU
classifier.add(Conv2D(32, (3, 3), activation = 'relu', padding = 'same'))

# Adding a maxpooling layer of size 2x2
classifier.add(MaxPooling2D(pool_size=(2,2)))

# Flattening the layers before the fully connected layer
classifier.add(Flatten())

# Adding fully connected layer with 512 neurons
classifier.add(Dense(units=512, activation='relu'))

# Adding a dropout with the probability of 0.5
classifier.add(Dropout(0.5))

# Adding a fully connected layer with 128 neurons
classifier.add(Dense(units=128,activation='relu'))

# The final output layer with 12 neurons to predict the categorical classification
classifier.add(Dense(units=12,activation='softmax'))

In [12]:
# Compiling the CNN classifier with Adam optimizer (default Learning rate and other parameters) and Categorical Crossentropy as loss function and Accuracy as the metric to monitor

opt= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, decay=0.001, epsilon=None,amsgrad=False)
classifier.compile(optimizer=opt,loss= 'categorical_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

In [13]:
# Dataset Pre-processing
# ImageDataGenerator is a powerful preprocessing utility to generate training and testing data with common data augmentation techniques. It can also be used to generate training data from Images stored in hierarchical directory structures.

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir=r"C:\Users\ac253\AINML\Computer Vision Week 3 (Project 1)\train"

# Create data generator for training data with data augmentation and normalizing all
# values by 255
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

# Setting training data generator's source directory
# Setting the target size to resize all the images to (64,64) as the model input layer expects 32X32 images

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    train_data_dir, # same directory as training data
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    subset='validation') # set as validation data


Found 3816 images belonging to 12 classes.
Found 951 images belonging to 12 classes.


In [15]:
classifier.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 32,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // 32,
    epochs = 20)

Epoch 1/20
119/119 [==============================] - 47s 362ms/step - loss: 2.4263 - accuracy: 0.1401 - val_loss: 2.3935 - val_accuracy: 0.2284
Epoch 2/20
119/119 [==============================] - 43s 359ms/step - loss: 2.0226 - accuracy: 0.3052 - val_loss: 1.6358 - val_accuracy: 0.4181
Epoch 3/20
119/119 [==============================] - 46s 384ms/step - loss: 1.6354 - accuracy: 0.4057 - val_loss: 1.4463 - val_accuracy: 0.4925
Epoch 4/20
119/119 [==============================] - 43s 359ms/step - loss: 1.4082 - accuracy: 0.5066 - val_loss: 1.3307 - val_accuracy: 0.5302
Epoch 5/20
119/119 [==============================] - 43s 359ms/step - loss: 1.2019 - accuracy: 0.5748 - val_loss: 1.1181 - val_accuracy: 0.6369
Epoch 6/20
119/119 [==============================] - 43s 363ms/step - loss: 1.0990 - accuracy: 0.6123 - val_loss: 1.0346 - val_accuracy: 0.6509
Epoch 7/20
119/119 [==============================] - 42s 357ms/step - loss: 1.0121 - accuracy: 0.6409 - val_loss: 0.9672 - val_ac

### In the 20th epoch we have managed to obtain a training accuracy of 0.8055 and validation accuracy of 0.8179

In [16]:
# Saving the model and its weights after training
classifier.save('./classifier.h5')

classifier.save_weights('./classifier_weights.h5')

In [17]:
print(os.listdir(cwd))

['.ipynb_checkpoints', 'classifier.h5', 'classifier_weights.h5', 'Computer vision project.ipynb', 'CV 1 - Project-1.pdf', 'Predict.png', 'Seedling - Prediction-20210824T073124Z-001.zip', 'train', 'train-20210824T073124Z-001.zip']


We can see that the classifier and its weights are saved in current directory.

In [18]:
# Let's test the model now
# Loading the pre-trained saved model

from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the pre trained model from the HDF5 file saved previously
pretrained_model = load_model('./classifier.h5')
pretrained_model.load_weights('./classifier_weights.h5')

Testing the model on a test image from one of the test folders

In [19]:
test_image = cv2.imread(r'C:\Users\ac253\AINML\Computer Vision Week 3 (Project 1)\Predict.png')
# Resize the image to 64X64 shape to be compatible with the model
test_image = cv2.resize(test_image,(64,64))

# Check if the size of the Image array is compatible with Keras model
print(test_image.shape)

# If not compatible expand the dimensions to match with the Keras Input
test_image = np.expand_dims(test_image, axis = 0)
test_image =test_image*1/255.0

#Check the size of the Image array again
print('After expand_dims: '+ str(test_image.shape))


#Predict the result of the test image
result = pretrained_model.predict(test_image)

# Check the indices Image Data Generator has allotted to each folder
classes_dict = train_generator.class_indices
print(classes_dict)

# Creating a list of classes in test set for showing the result as the folder name
prediction_class = []
for class_name,index in classes_dict.items():
  prediction_class.append(class_name)
  
print(result[0])

# Index of the class with maximum probability
predicted_index = np.argmax(result[0])

# Print the name of the class
print(prediction_class[predicted_index])

(64, 64, 3)
After expand_dims: (1, 64, 64, 3)
{'Black-grass': 0, 'Charlock': 1, 'Cleavers': 2, 'Common Chickweed': 3, 'Common wheat': 4, 'Fat Hen': 5, 'Loose Silky-bent': 6, 'Maize': 7, 'Scentless Mayweed': 8, 'Shepherds Purse': 9, 'Small-flowered Cranesbill': 10, 'Sugar beet': 11}
[5.19503196e-08 9.21515405e-01 3.57701890e-02 4.07363956e-08
 1.37994948e-06 3.25634405e-02 2.20534417e-08 8.47028848e-03
 2.68357748e-04 1.15911644e-04 1.26988476e-03 2.52227310e-05]
Charlock


### Comparing the results from above algorithms we can say that the CNN has better training accuracy and validation accuracy than the Neural Network. 
In CNN we used 64x64 images to train the classifier. Where as we used only 16x16 images to train the Neural Network. Using higher dimension input images would have created a lot of trainable parameters for such a huge dataset. Thus there would have a lack of memory to train such huge set of parameters or it would have take a lot time to train over my RTX 2060. Since neural networks are very hardware dependent.
CNN on the other hand does not have a large number of trainable parameters compared to neural networks. CNNs are also robust when it comes to image recognition. 